# Майнор ВШЭ 

## Прикладные задачи анализа данных 2020

## Семинар 1: Рекомендательные системы

На этом семинаре мы рассмотрим основные подходы к задаче рекомендации и увидим, как это выглядит на практике на небольшом датасете.

Воспользуемся датасетом: Articles Sharing and Reading from CI&T Deskdrop.

In [195]:
%pylab inline

import pandas as pd
from tqdm import tqdm_notebook

## Часть 0. Загрузка данных

Загрузим [Deskdrop dataset](https://www.dropbox.com/s/p69j0d978tetktj/articles-sharing-reading-from-cit-deskdrop.zip), включающийся в себе логи за 1 год платформы, где пользователи читают статьи.

Данные включают в себя 2 файла:  
- **shared_articles.csv**
- **users_interactions.csv**

Как можно догадаться, в одном описания самих статей (нам понадобятся в контентных моделях), а в другом логи пользователей.

In [79]:
# загружаем и разархивируем данные, для вашего удобства данные можно найти по ссылке:
# https://www.dropbox.com/s/p69j0d978tetktj/articles-sharing-reading-from-cit-deskdrop.zip
# в Colab и linux вам могут пригодиться утилиты wget и unzip
# такж, если вы работаете в Google Colab, устанавливаем библиотеки, которые потребуеются дальше: pyfm, catboost
####### Здесь ваш код ##########
raise NotImplementedError
################################

#### shared_articles.csv

Так как в файле перечислены даже удалённые статьи, то мы их сразу удалим (на самом деле они могли бы быть нам полезны для подсчёта различных величин, хоть мы и не можем их рекомендовать).

In [218]:
articles_df = pd.read_csv("shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

#### users_interactions.csv

В колонке eventType описаны действия, которые могли совершать пользователи над статьёй:  
- VIEW
- LIKE
- COMMENT CREATED
- FOLLOW
- BOOKMARK

In [219]:
interactions_df = pd.read_csv("users_interactions.csv")
interactions_df.head(10)

In [220]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

### Предобработка данных

В логах встречаются различные действия пользователей. Однако мы хотим работать лишь с одной величиной, характеризующей всё взаимодействие пользователя со статьёй. Предлагается задать действиям следующие веса:

In [221]:
event_type_strength = {
    "VIEW": 1.0,
    "LIKE": 2.0,
    "BOOKMARK": 2.5,
    "FOLLOW": 3.0,
    "COMMENT CREATED": 4.0,
}

Посчитаем числовую величину "оценки" пользователем статьи с указанными выше весами.

In [222]:
# interactions_df['eventStrength'] =
####### Здесь ваш код ##########
raise NotImplementedError
################################
interactions_df["eventStrength"]

Ремендательные системы подвержены проблеме холодного старта. Будем работать только с теми пользователями, которые взаимодействовали хотя бы с 5-ю материалами.

Оставим только таких пользователей.

In [223]:
users_interactions_count_df = (
    interactions_df.groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId")
    .size()
)
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = users_interactions_count_df[
    users_interactions_count_df >= 5
].reset_index()[["personId"]]
print("# users with at least 5 interactions:", len(users_with_enough_interactions_df))

In [224]:
users_interactions_count_df.hist(bins=5)

Оставим только те взаимодействия, которые касаются только отфильтрованных пользователей.

In [225]:
# interactions_from_selected_users_df =
####### Здесь ваш код ##########
raise NotImplementedError
################################

In [226]:
print("# interactions before:", interactions_df.shape)
print("# interactions after:", interactions_from_selected_users_df.shape)

В данной постановке каждый пользователей мог взаимодействовать с каждой статьёй более 1 раза (как минимум совершая различные действия). Предлагается "схлопнуть" все действия в одно взаимодействие с весом, равным сумме весов. 

Однако полученное число будет в том числе тем больше, чем больше действий произвёл человек. Чтобы уменьшить разброс предлагается взять логарифм от полученного числа (можно придумывать другие веса действиям и по-другому обрабатывать значения).

Также сохраним последнее значение времени взаимодействия для разделениея выборки на обучение и контроль.

In [227]:
def smooth_user_preference(x):
    return math.log(1 + x, 2)


interactions_full_df = (
    interactions_from_selected_users_df.groupby(["personId", "contentId"])
    .eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index()
    .set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = interactions_from_selected_users_df.groupby(
    ["personId", "contentId"]
)["timestamp"].last()

interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(20)

Разобьём выборку на обучение и контроль по времени.

In [279]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp < split_ts
].copy()
interactions_test_df = interactions_full_df.loc[
    interactions_full_df.last_timestamp >= split_ts
].copy()

print("# interactions on Train set: %d" % len(interactions_train_df))
print("# interactions on Test set: %d" % len(interactions_test_df))

interactions_train_df

Также разбить данные можно не опираясь на временной фактор:
```python
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.25,
                                   random_state=42)

```
Как повлият такое разбиение на итоговый результат?

Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказанями в виде списков.

In [280]:
interactions = (
    interactions_train_df.groupby("personId")["contentId"]
    .agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = interactions_test_df.groupby("personId")["contentId"].agg(
    lambda x: list(x)
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    list()
    for x in range(
        len(interactions.loc[pd.isnull(interactions.true_test), "true_test"])
    )
]

interactions.head(1)

## Часть 1: Baseline (модель по популярности)

Самой простой моделью рекомендаций (при этом достаточно сильной!) является модель, которая рекомендует наиболее популярные предметы. 

Реализуем её. Давайте считать, что рекомендуем мы по 10 материалов (такое ограничение на размер блока на сайте).

Посчитаем популярность каждой статьи, как сумму всех "оценок" взаимодействий с ней. Отсортируем материалы по их популярности.

In [281]:
# popular_content =
####### Здесь ваш код ##########
raise NotImplementedError
################################

In [282]:
print(articles_df.loc[articles_df.contentId == popular_content[2]]["title"].values)

In [283]:
print(articles_df.loc[articles_df.contentId == popular_content[2363]]["title"].values)

Теперь необходимо сделать предсказания для каждого пользователя. Не забываем, что надо рекомендовать то, что пользователь ещё не читал (для этого нужно проверить, что материал не встречался в true_train).

In [284]:
top_k = 10

# interactions['prediction_popular'] =
####### Здесь ваш код ##########
raise NotImplementedError
################################
interactions["prediction_popular"][0]

Настало время оценить качество. Посчитаем precision@10 для каждого пользователя (доля угаданных рекомендаций). Усредним по всем пользователям. Везде далее будем считать эту же метрику.

In [285]:
def calc_precision(column):
    ####### Здесь ваш код ##########
    raise NotImplementedError
    ################################

In [286]:
calc_precision("prediction_popular")

## Часть 2. Коллаборативная фильтрация.

Перейдём к более сложному механизму рекомендаций, а именно коллаборативной фильтрации. Суть коллаборативной фильтрации в том, что учитывается схожесть пользователей и товаров между собой, а не факторы, которые их описывают. 

Для начала для удобства составим матрицу "оценок" пользователей. Нули будут обозначать отсутствие взаимодействия.

In [287]:
ratings = pd.pivot_table(
    interactions_train_df, values="eventStrength", index="personId", columns="contentId"
).fillna(0)

### Memory-based

Посчитаем схожести пользователей с помощью корреляции Пирсона. Для каждой пары учитываем только ненулевые значения.

Для скорости работы лучше переходить от pandas к numpy.

In [288]:
ratings_m = ratings.values

In [289]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))

for i in tqdm_notebook(range(len(ratings_m) - 1)):
    for j in range(i + 1, len(ratings_m)):

        # nonzero elements of two users
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)

        # continue if no intersection
        if np.sum(mask_uv) == 0:
            continue

        # get nonzero elements
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        # for nonzero std
        if len(np.unique(ratings_v)) < 2 or len(np.unique(ratings_u)) < 2:
            continue
        # similarity_users[i,j] =
        # similarity_users[j,i] =
        ####### Здесь ваш код ##########
        raise NotImplementedError
        ################################

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для рекомендаций.

Для каждого пользователя:

1. Найдём пользователей с похожестью больше $\alpha$ на нашего пользователя.
2. Посчитаем для каждой статьи долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
3. Порекомендуем статьи с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

В нашем примере данных не очень много, поэтому возьмём $\alpha = 0$.

После того, как будут сделаны предсказания (новый столбец в interactions), посчитаем качество по той же метрике.

In [290]:
prediction_user_based = []
####### Здесь ваш код ##########
raise NotImplementedError
################################
interactions["prediction_user_based"] = prediction_user_based

In [292]:
calc_precision("prediction_user_based")

### Модель со скрытыми переменными

Реализуем подход с разложением матрицы оценок. Для этого сделаем сингулярное разложение (svd в scipy.linalg), на выходе вы получите три матрицы.

Заметим, что мы используем матрицу с нулями, будто отсутствующие взаимодействия негативные, что странно.

Если бы мы учили модель со скрытыми переменными с помощью стохастического градиентного спуска, то неизвестные взаимодействия могли бы не использовать.

In [293]:
from scipy.linalg import svd

In [294]:
####### Здесь ваш код ##########
raise NotImplementedError
################################

In [295]:
####### Здесь ваш код ##########
raise NotImplementedError
################################

Значения у матрицы с сингулярными числами отсортированы по убыванию. Допустим мы хотим оставить только первые 100 компонент (и получить скрытые представления размерности 100). Для этого необходимо оставить 100 столбцов в матрице U, оставить из sigma только первые 100 значений (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Перемножим преобразованные матрицы ($\hat{U}, \hat{sigma}, \hat{V^T}$), чтобы получить восстановленную матрицу оценок.

In [296]:
K = 100
####### Здесь ваш код ##########
raise NotImplementedError
################################

Посчитаем качество аппроксимации матрицы по норме Фробениуса (среднеквадратичную ошибку между всеми элементами соответствующими элементами двух матриц). Сравним его с простым бейзлайном с константным значением, равным среднему значению исходной матрицы. У аппроксимации ошибка должна получиться ниже.

In [297]:
####### Здесь ваш код ##########
raise NotImplementedError
################################

Теперь можно делать предсказания по матрице. Сделаем их (не забываем про то, что уже было просмотрено пользователем), оценим качество. Для этого необходимо для каждого пользователя найти предметы с наибольшими оценками в восстановленной матрице.

In [298]:
new_ratings = pd.DataFrame(new_ratings, index=ratings.index, columns=ratings.columns)

predictions = []
####### Здесь ваш код ##########
raise NotImplementedError
################################
interactions["prediction_svd"] = predictions

In [299]:
calc_precision("prediction_svd")

## Часть 3. Контентные  модели

В этой части реализуем альтернативных подход к рекомендательным системам — контентные модели.

Теперь мы будем оперировать не матрицей с оценками, а классической для машинного обучения матрицей объекты-признаки. Каждый объект будет характеризовать пару user-item и содержать признаки, описывающие как пользователя, так и товар. Кроме этого признаки могут описывать и саму пару целиком.

Матрица со всеми взаимодействиями уже получена нами на этапа разбиения выборки на 2 части. 

Будем обучать классификатор на взаимодействие, а для него нужны отрицательные примеры. Добавим случайные отсутствующие взаимодействия как отрицательные.

Заметим, что модель оценивает каждую пару потенциального взаимодействия, а значит, надо подготовить выборку из всех возможных пар из пользователей и статей.

In [300]:
test_personId = np.repeat(interactions.index, len(ratings.columns))
test_contentId = list(ratings.columns) * len(interactions)
test = pd.DataFrame(
    np.array([test_personId, test_contentId]).T, columns=["personId", "contentId"]
)

interactions_train_df = pd.concat(
    (
        interactions_train_df,
        test.loc[np.random.permutation(test.index)[: 4 * len(interactions_train_df)]],
    ),
    ignore_index=True,
)
interactions_train_df.eventStrength.fillna(0, inplace=True)

Придумаем и добавим признаков о пользователях и статьях. Сначала добавим информацию о статьях в данные о взаимодействиях.

In [301]:
interactions_train_df = interactions_train_df.merge(
    articles_df, how="left", on="contentId"
)
interactions_test_df = interactions_test_df.merge(
    articles_df, how="left", on="contentId"
)

In [302]:
# first feature index
features_start = len(interactions_train_df.columns)

После обучения модели нам придётся делать предсказания на тестовой выборке для всех возможных пар статья-пользователь. Подготовим такую матрицу, чтобы параллельно посчитать признаки для неё.

In [303]:
test_personId = np.repeat(interactions.index, len(articles_df))
test_contentId = list(articles_df.contentId) * len(interactions)
test = pd.DataFrame(
    np.array([test_personId, test_contentId]).T, columns=["personId", "contentId"]
)
test = test.merge(articles_df, how="left", on="contentId")

test.head()

Добавим признаки-индикаторы возможных значений contentType.

In [304]:
interactions_train_df["is_HTML"] = interactions_train_df.contentType == "HTML"
####### Здесь ваш код ##########
raise NotImplementedError
################################

Добавим признаки "длина названия" и "длина текста" + некоторые проверки на ключевые слова.

In [305]:
interactions_train_df["title_length"] = interactions_train_df.title.fillna("").apply(
    len
)
####### Здесь ваш код ##########
raise NotImplementedError
################################

Добавим признаки-индикаторы языка.

In [306]:
interactions_train_df["is_lang_en"] = interactions_train_df.lang == "en"
####### Здесь ваш код ##########
raise NotImplementedError
################################

Обучим на полученных признаках градиентный бустинг.

In [307]:
import catboost

model = catboost.CatBoostClassifier()
model.fit(
    interactions_train_df[interactions_train_df.columns[features_start:]],
    np.array(interactions_train_df.eventStrength > 0, dtype=int),
)

Сделаем предсказания на тестовой выборке, сформируем из них рекомендации. 

In [308]:
####### Здесь ваш код ##########
raise NotImplementedError
################################

Оценим их качество.

In [309]:
# calc_precision(...)
####### Здесь ваш код ##########
raise NotImplementedError
################################

## Часть 4. Факторизационная машина

Вспомним, что факторизационная машина учитывает попарные взаимодействия признаков, что приводит сразу и к использованию контента (сами признаки), и к обучению скрытых представлений (индикаторы пользователей и статей).

Попробуем факторизационные машины из библиотеки pyFM (так как можно работать прямо из питона). https://github.com/coreylynch/pyFM

In [310]:
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer

Перейдём к обобщению матричных разложений — факторизационным машинам, которые работают могут работать с контентной информацией. Вспомним, какие данные у нас изначально были:

В факторизационную машину можно загрузить "айдишники" пользователей и статей (то есть сделать аналог коллаборативной фильтрации) и одновременно различные признаки.

Удобно обрабатывать категориальные переменные (id и другие) можно с помощью DictVectorizer. Например, процесс может выглядить вот так:
```python
train = [
    {"user": "1", "item": "5", "age": 19},
    {"user": "2", "item": "43", "age": 33},
    {"user": "3", "item": "20", "age": 55},
    {"user": "4", "item": "10", "age": 20},
]
v = DictVectorizer()
X = v.fit_transform(train)
y = np.repeat(1.0, X.shape[0])
fm = pylibfm.FM()
fm.fit(X,y)
fm.predict(v.transform({"user": "1", "item": "10", "age": 24}))
```

Сгенерируем таблицу с признаками в таком виде, где будут id пользователя, статьи и автора статьи и несколько признаков, которые вы сможете придумать.

In [311]:
train_data = []

for i in tqdm_notebook(range(len(interactions_train_df))):
    features = {}
    features["personId"] = str(interactions_train_df.iloc[i].personId)
    features["contentId"] = str(interactions_train_df.iloc[i].contentId)
    ####### Здесь ваш код ##########
    raise NotImplementedError
    ################################
    train_data.append(features)

Повторим эту процедуру для тестовой выборки. Заметим, что модель оценивает каждую пару потенциального взаимодействия, а значит, надо подготовить выборку из всех возможных пар из пользователей и статей.

In [312]:
from copy import deepcopy

test_data = []

for i in tqdm_notebook(range(len(interactions))):
    features = {}
    features["personId"] = str(interactions.index[i])
    for j in range(len(ratings.columns)):
        ####### Здесь ваш код ##########
        raise NotImplementedError
        ################################
        test_data.append(deepcopy(features))

Векторизуем, получим разреженные матрицы.

Мы будем обучать регрессор на силу взаимодействия, а для него нужны отрицательные примеры. Добавим некоторое количество случайных примеров как негативные (матрица взаимодействий разреженная, поэтому шансы взять как негативное взаимодействие некоторое положительное мало).

In [313]:
dv = DictVectorizer()

train_features = dv.fit_transform(
    train_data + list(np.random.permutation(test_data)[:100000])
)
test_features = dv.transform(test_data)

In [314]:
train_features

In [315]:
y_train = list(interactions_train_df.eventStrength.values) + list(np.zeros(100000))

Укажем размером скрытого представления 10, сделаем 10 итераций.

In [316]:
# fm = pylibfm.FM(...
####### Здесь ваш код ##########
raise NotImplementedError
################################
fm.fit(train_features, y_train)

Предскажем и оценим качество.

In [317]:
####### Здесь ваш код ##########
raise NotImplementedError
################################

In [320]:
predictions = []

for i, person in enumerate(interactions.index):
    user_prediction = ratings.columns[np.argsort(new_ratings[i])[::-1]]
    predictions.append(
        user_prediction[
            ~np.in1d(user_prediction, interactions.loc[person, "true_train"])
        ][:top_k]
    )

interactions["fm_prediction"] = predictions

In [321]:
calc_precision("fm_prediction")